# Day19_4: 오픈소스 LLM 파인튜닝 (PEFT & QLoRA) - 정답

## 실습 퀴즈 정답

---

In [ ]:
# 필요한 라이브러리 임포트
import torch
import numpy as np
import pandas as pd
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

---

### Q1. PEFT 개념 이해 (기본)

**문제**: 다음 빈칸을 채우세요.

1. PEFT는 (______)의 약자로, 전체 모델이 아닌 아주 작은 부분만 학습합니다.
2. LoRA는 (______)을 통해 원본 가중치 행렬을 근사합니다.
3. QLoRA는 (______)비트 양자화와 LoRA를 결합한 기술입니다.

In [ ]:
# 정답
answer1 = "Parameter-Efficient Fine-Tuning"
answer2 = "저차원 행렬 분해 (Low-Rank Decomposition)"
answer3 = "4"

print(f"1. PEFT = {answer1}")
print(f"   (매개변수 효율적 파인튜닝)")
print()
print(f"2. LoRA는 {answer2}을 통해 원본 가중치를 근사합니다.")
print(f"   W' = W + BA (B: d x r, A: r x k, r << d,k)")
print()
print(f"3. QLoRA는 {answer3}비트 양자화 + LoRA를 결합합니다.")
print(f"   4비트 NormalFloat(NF4)를 사용하여 메모리를 1/4로 절약")

**풀이 설명**:
- **PEFT**: Parameter-Efficient Fine-Tuning의 약자로, 모델의 일부 파라미터만 학습하여 효율성을 높이는 기법입니다.
- **LoRA**: Low-Rank Adaptation의 약자로, 큰 행렬을 두 개의 작은 행렬의 곱으로 분해하여 학습 파라미터를 줄입니다.
- **QLoRA**: Quantized LoRA로, 4비트 양자화를 통해 모델 가중치를 압축하고 LoRA를 적용합니다.

---

### Q2. LoRA 파라미터 비율 계산 (기본)

**문제**: 다음 조건에서 LoRA 학습 파라미터 비율을 계산하세요.

- 원래 가중치 행렬: 4096 x 4096
- LoRA rank (r): 8

In [ ]:
# 정답
d = 4096  # 입력 차원
k = 4096  # 출력 차원
r = 8     # LoRA rank

# 원래 파라미터 수
original_params = d * k

# LoRA 파라미터 수
# B: (d x r) + A: (r x k)
lora_params = (d * r) + (r * k)

# 비율 계산
ratio = (lora_params / original_params) * 100

print("LoRA 파라미터 비율 계산")
print("=" * 50)
print(f"원래 가중치 행렬: {d} x {k}")
print(f"LoRA rank (r): {r}")
print()
print(f"원래 파라미터 수: {original_params:,}")
print(f"LoRA 파라미터 수: {lora_params:,}")
print(f"  - B 행렬 (d x r): {d} x {r} = {d * r:,}")
print(f"  - A 행렬 (r x k): {r} x {k} = {r * k:,}")
print()
print(f"학습 파라미터 비율: {ratio:.4f}%")
print(f"메모리 절약: {100 - ratio:.2f}%")

**풀이 설명**:
- LoRA는 원본 가중치 W (d x k)를 BA로 근사합니다.
- B: (d x r), A: (r x k)이므로 총 파라미터는 r(d + k)입니다.
- r=8, d=k=4096일 때: 8 x (4096 + 4096) = 65,536개
- 원본 대비: 65,536 / 16,777,216 = 약 0.39%

---

### Q3. 양자화 메모리 계산 (기본)

**문제**: 7B 파라미터 모델의 메모리 사용량을 비교하세요.

In [ ]:
# 정답
params = 7_000_000_000  # 7B 파라미터

# 각 정밀도별 바이트
precision = {
    "FP32 (32비트)": 4,    # 4 bytes
    "FP16 (16비트)": 2,    # 2 bytes
    "INT8 (8비트)": 1,     # 1 byte
    "INT4 (4비트)": 0.5    # 0.5 bytes
}

print("7B 파라미터 모델 메모리 사용량 (가중치만)")
print("=" * 50)

for name, bytes_per_param in precision.items():
    memory_bytes = params * bytes_per_param
    memory_gb = memory_bytes / (1024 ** 3)  # bytes to GB
    print(f"{name}: {memory_gb:.2f} GB")

print()
print("결론:")
print("- FP32 -> INT4: 메모리 1/8로 감소 (28GB -> 3.5GB)")
print("- QLoRA 덕분에 일반 GPU(8GB VRAM)에서도 7B 모델 파인튜닝 가능!")

**풀이 설명**:
- 메모리 = 파라미터 수 x 파라미터당 바이트
- FP32: 4바이트, FP16: 2바이트, INT8: 1바이트, INT4: 0.5바이트
- 양자화를 통해 메모리를 크게 절약할 수 있습니다.

---

### Q4. 토크나이저 사용 (응용)

**문제**: 아래 텍스트를 Llama 3 Instruct 템플릿으로 변환하세요.

In [ ]:
# 정답
def create_llama3_prompt(system_message, user_message):
    """
    Llama 3 Instruct 템플릿으로 프롬프트 생성
    
    Parameters:
    - system_message: 시스템 메시지
    - user_message: 사용자 메시지
    
    Returns:
    - 포맷팅된 프롬프트
    """
    if system_message:
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    else:
        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    return prompt

# 테스트
system_msg = "당신은 데이터 분석 전문가입니다."
user_msg = "pandas와 numpy의 차이점은?"

prompt = create_llama3_prompt(system_msg, user_msg)

print("Llama 3 Instruct 템플릿:")
print("=" * 60)
print(prompt)

**풀이 설명**:
- Llama 3 Instruct 모델은 특수 토큰을 사용한 템플릿을 따릅니다.
- `<|begin_of_text|>`: 텍스트 시작
- `<|start_header_id|>`: 역할 헤더 시작
- `<|end_header_id|>`: 역할 헤더 끝
- `<|eot_id|>`: 턴 종료

---

### Q5. LoRA 설정 변경 (응용)

**문제**: 다음 요구사항에 맞는 LoraConfig를 작성하세요.

In [ ]:
from peft import LoraConfig

# 정답
lora_config = LoraConfig(
    r=32,                           # rank
    lora_alpha=64,                  # alpha (r의 2배)
    lora_dropout=0.1,               # dropout
    bias="none",                    # bias 학습 안함
    task_type="CAUSAL_LM",          # 언어 모델 태스크
    target_modules=[                # 모든 Linear 레이어
        "q_proj",     # Query projection
        "k_proj",     # Key projection
        "v_proj",     # Value projection
        "o_proj",     # Output projection
        "gate_proj",  # MLP gate
        "up_proj",    # MLP up
        "down_proj"   # MLP down
    ]
)

print("LoRA 설정:")
print("=" * 50)
print(f"r (rank): {lora_config.r}")
print(f"lora_alpha: {lora_config.lora_alpha}")
print(f"lora_dropout: {lora_config.lora_dropout}")
print(f"target_modules: {lora_config.target_modules}")
print()
print("적용 레이어:")
print("  - Attention: q_proj, k_proj, v_proj, o_proj")
print("  - MLP/FFN: gate_proj, up_proj, down_proj")

**풀이 설명**:
- `r=32`: 더 높은 rank로 표현력 증가 (메모리 사용량도 증가)
- `lora_alpha=64`: 일반적으로 r의 2배로 설정
- `target_modules`: Llama 모델의 모든 Linear 레이어에 LoRA 적용
  - Attention: Query, Key, Value, Output projection
  - MLP: Gate, Up, Down projection

---

### Q6. 학습 인자 최적화 (응용)

**문제**: 메모리 제한(8GB VRAM)에서 안정적인 학습을 위한 TrainingArguments를 작성하세요.

In [ ]:
from transformers import TrainingArguments

# 정답
training_args = TrainingArguments(
    output_dir="./output",
    
    # 배치 설정 (메모리 최적화)
    per_device_train_batch_size=1,     # 최소 배치 (메모리 절약)
    gradient_accumulation_steps=8,     # 효과적 배치 크기 = 1 x 8 = 8
    
    # 학습률 설정
    learning_rate=2e-4,                # QLoRA는 높은 학습률 가능
    warmup_ratio=0.1,                  # 10% warmup
    
    # 에포크 및 스텝
    num_train_epochs=2,                # 2 에포크
    max_steps=-1,                      # 에포크 기반 학습
    
    # 메모리 최적화
    fp16=True,                         # FP16 사용
    gradient_checkpointing=True,       # 그래디언트 체크포인팅
    
    # 로깅
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    
    # 옵티마이저
    optim="adamw_torch",
    weight_decay=0.01,
)

print("메모리 최적화 학습 설정:")
print("=" * 50)
print(f"배치 크기: {training_args.per_device_train_batch_size}")
print(f"그래디언트 누적: {training_args.gradient_accumulation_steps}")
print(f"효과적 배치 크기: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"학습률: {training_args.learning_rate}")
print(f"에포크: {training_args.num_train_epochs}")
print(f"FP16: {training_args.fp16}")
print(f"Gradient Checkpointing: {training_args.gradient_checkpointing}")
print()
print("메모리 절약 전략:")
print("  1. 배치 크기 1 + 그래디언트 누적 8")
print("  2. FP16 학습")
print("  3. Gradient Checkpointing (활성화 재계산)")

**풀이 설명**:
- **배치 크기 1**: 메모리 사용량 최소화
- **gradient_accumulation_steps=8**: 작은 배치를 누적하여 효과적 배치 크기 8 달성
- **gradient_checkpointing**: 활성화 값을 저장하지 않고 필요시 재계산하여 메모리 절약
- **fp16**: 16비트 부동소수점으로 메모리와 속도 최적화

---

### Q7. 데이터셋 포맷팅 함수 (복합)

**문제**: 다양한 형식의 데이터를 Llama 3 템플릿으로 변환하는 함수를 작성하세요.

In [ ]:
# 정답
def format_to_llama3(example):
    """
    다양한 형식의 데이터를 Llama 3 템플릿으로 변환
    
    지원 형식:
    1. {"question": ..., "answer": ...} - Q&A 형식
    2. {"instruction": ..., "input": ..., "output": ...} - Alpaca 형식
    3. {"prompt": ..., "response": ...} - 일반 형식
    
    Returns:
    - Llama 3 Instruct 형식의 텍스트
    """
    # 형식 1: Q&A
    if "question" in example and "answer" in example:
        user_content = example["question"]
        assistant_content = example["answer"]
    
    # 형식 2: Alpaca (instruction + input + output)
    elif "instruction" in example and "output" in example:
        if example.get("input"):
            user_content = f"{example['instruction']}\n\n입력: {example['input']}"
        else:
            user_content = example["instruction"]
        assistant_content = example["output"]
    
    # 형식 3: 일반 (prompt + response)
    elif "prompt" in example and "response" in example:
        user_content = example["prompt"]
        assistant_content = example["response"]
    
    else:
        raise ValueError(f"지원하지 않는 형식: {example.keys()}")
    
    # Llama 3 템플릿으로 변환
    formatted = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{assistant_content}<|eot_id|>"""
    
    return formatted

# 테스트
test_examples = [
    {"question": "머신러닝이란?", "answer": "데이터로부터 학습하는 AI입니다."},
    {"instruction": "번역하세요", "input": "Hello", "output": "안녕하세요"},
    {"prompt": "파이썬의 장점", "response": "읽기 쉽고 배우기 쉽습니다."}
]

print("다양한 형식 -> Llama 3 템플릿 변환:")
print("=" * 60)
for i, ex in enumerate(test_examples, 1):
    print(f"\n예시 {i} - 입력 형식: {list(ex.keys())}")
    print("-" * 40)
    print(format_to_llama3(ex))
    print()

**풀이 설명**:
- 입력 딕셔너리의 키를 확인하여 형식을 판별합니다.
- Alpaca 형식은 instruction과 input을 조합하여 user 메시지로 만듭니다.
- 모든 형식을 동일한 Llama 3 템플릿으로 변환합니다.

---

### Q8. 추론 함수 개선 (복합)

**문제**: 생성 파라미터를 조절할 수 있는 개선된 추론 함수를 작성하세요.

In [ ]:
# 정답
def generate_with_params(
    model, 
    tokenizer, 
    question,
    system_prompt=None,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    max_new_tokens=200
):
    """
    파라미터 조절 가능한 추론 함수
    
    Parameters:
    - model: 모델
    - tokenizer: 토크나이저
    - question: 사용자 질문
    - system_prompt: 시스템 프롬프트 (선택)
    - temperature: 창의성 조절 (0~2)
    - top_p: nucleus sampling
    - top_k: top-k sampling
    - max_new_tokens: 최대 생성 토큰
    
    Returns:
    - dict: {'answer': str, 'input_tokens': int, 'output_tokens': int}
    """
    # 프롬프트 구성
    if system_prompt:
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    else:
        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    # 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_length = inputs["input_ids"].shape[1]
    
    # 생성
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True if temperature > 0 else False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    output_length = outputs.shape[1]
    new_tokens = output_length - input_length
    
    # 디코딩
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 답변 부분만 추출
    if "assistant" in full_response.lower():
        parts = full_response.split("assistant")
        answer = parts[-1].strip() if len(parts) > 1 else full_response
    else:
        answer = full_response
    
    return {
        "answer": answer,
        "input_tokens": input_length,
        "output_tokens": new_tokens,
        "total_tokens": output_length
    }

# 함수 설명
print("generate_with_params 함수 사용법:")
print("=" * 60)
print("""
result = generate_with_params(
    model, tokenizer, 
    "파이썬의 장점은?",
    system_prompt="간결하게 답변하세요.",
    temperature=0.3,  # 낮으면 결정적
    top_p=0.9,        # nucleus sampling
    top_k=50          # top-k sampling
)

print(result['answer'])
print(f"토큰: {result['input_tokens']} -> {result['output_tokens']}")
""")

**풀이 설명**:
- `temperature`: 0에 가까울수록 결정적, 높을수록 다양한 출력
- `top_p`: 누적 확률이 p 이하인 토큰만 샘플링
- `top_k`: 상위 k개 토큰만 샘플링 대상으로
- 토큰 수를 반환하여 비용/성능 분석에 활용

---

### Q9. 전체 파인튜닝 파이프라인 (종합)

**문제**: CSV 파일에서 데이터를 로드하고 파인튜닝하는 전체 파이프라인을 작성하세요.

In [ ]:
# 정답
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from trl import SFTTrainer
import pandas as pd

def finetune_from_csv(csv_path, model_id, output_dir, 
                       lora_r=16, lora_alpha=32, epochs=3):
    """
    CSV 데이터로 LLM 파인튜닝 파이프라인
    
    Parameters:
    - csv_path: Q&A CSV 파일 경로 (question, answer 컬럼 필요)
    - model_id: Hugging Face 모델 ID
    - output_dir: 결과 저장 경로
    - lora_r: LoRA rank
    - lora_alpha: LoRA alpha
    - epochs: 학습 에포크
    
    Returns:
    - trainer: 구성된 SFTTrainer
    """
    print("="*60)
    print("LLM 파인튜닝 파이프라인")
    print("="*60)
    
    # 1. 데이터 로드
    print("\n[1/6] 데이터 로드...")
    df = pd.read_csv(csv_path)
    print(f"  - 로드된 샘플 수: {len(df)}")
    
    # 2. 데이터셋 변환
    print("\n[2/6] 데이터셋 변환...")
    dataset = Dataset.from_pandas(df[["question", "answer"]])
    
    # 3. 템플릿 포맷팅
    print("\n[3/6] 템플릿 포맷팅...")
    def format_prompt(example):
        return {
            "text": f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{example['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{example['answer']}<|eot_id|>"""
        }
    
    formatted_dataset = dataset.map(format_prompt)
    print(f"  - 포맷팅 완료")
    
    # 4. 모델 및 토크나이저 로드
    print(f"\n[4/6] 모델 로드: {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    
    # GPU/CPU에 따라 양자화 설정
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    if device == "cuda":
        from transformers import BitsAndBytesConfig
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto"
        )
        print("  - GPU 환경: 4비트 양자화 적용")
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float32,
            device_map="auto"
        )
        print("  - CPU 환경: 양자화 없이 로드")
    
    # 5. LoRA 설정
    print(f"\n[5/6] LoRA 설정 (r={lora_r}, alpha={lora_alpha})")
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    )
    
    # 6. Trainer 구성
    print(f"\n[6/6] SFTTrainer 구성...")
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=epochs,
        logging_steps=10,
        save_total_limit=2,
        fp16=(device == "cuda"),
        warmup_ratio=0.1
    )
    
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=formatted_dataset,
        peft_config=lora_config,
        tokenizer=tokenizer
    )
    
    print("\n" + "="*60)
    print("파이프라인 구성 완료!")
    print("학습 시작: trainer.train()")
    print("="*60)
    
    return trainer

# 사용 예시 (실제 CSV 파일 필요)
print("사용 예시:")
print("""
# CSV 파일 준비 (question, answer 컬럼)
# qa_data.csv:
# question,answer
# "EDA란?","탐색적 데이터 분석입니다."
# ...

trainer = finetune_from_csv(
    csv_path="data/qa_dataset.csv",
    model_id="meta-llama/Llama-3.2-1B-Instruct",
    output_dir="./output",
    lora_r=16,
    epochs=3
)

# 학습 실행
trainer.train()

# 어댑터 저장
trainer.save_model("./my-adapter")
""")

**풀이 설명**:
- 6단계로 구성된 파인튜닝 파이프라인입니다.
- CSV 로드 -> 데이터셋 변환 -> 템플릿 포맷팅 -> 모델 로드 -> LoRA 설정 -> Trainer 구성
- GPU/CPU 환경을 자동 감지하여 양자화를 적용합니다.

---

### Q10. 모델 비교 평가 (종합)

**문제**: 파인튜닝 전후 모델의 응답을 비교 평가하는 함수를 작성하세요.

In [ ]:
# 정답
import pandas as pd

def compare_models(base_model, finetuned_model, tokenizer, test_questions, keywords=None):
    """
    파인튜닝 전후 모델 비교
    
    Parameters:
    - base_model: 파인튜닝 전 모델
    - finetuned_model: 파인튜닝 후 모델  
    - tokenizer: 토크나이저
    - test_questions: 테스트 질문 리스트
    - keywords: 확인할 키워드 딕셔너리 {질문: [키워드들]} (선택)
    
    Returns:
    - pd.DataFrame: 비교 결과
    """
    results = []
    
    for question in test_questions:
        # 프롬프트 구성
        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
        inputs = tokenizer(prompt, return_tensors="pt")
        
        # 기본 모델 응답
        base_model.eval()
        with torch.no_grad():
            base_outputs = base_model.generate(
                inputs["input_ids"].to(base_model.device),
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        base_response = tokenizer.decode(base_outputs[0], skip_special_tokens=True)
        
        # 파인튜닝 모델 응답
        finetuned_model.eval()
        with torch.no_grad():
            ft_outputs = finetuned_model.generate(
                inputs["input_ids"].to(finetuned_model.device),
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        ft_response = tokenizer.decode(ft_outputs[0], skip_special_tokens=True)
        
        # 답변 부분 추출
        def extract_answer(response):
            if "assistant" in response.lower():
                return response.split("assistant")[-1].strip()
            return response
        
        base_answer = extract_answer(base_response)
        ft_answer = extract_answer(ft_response)
        
        # 메트릭 계산
        result = {
            "question": question[:50] + "..." if len(question) > 50 else question,
            "base_length": len(base_answer),
            "ft_length": len(ft_answer),
            "base_answer": base_answer[:100] + "..." if len(base_answer) > 100 else base_answer,
            "ft_answer": ft_answer[:100] + "..." if len(ft_answer) > 100 else ft_answer
        }
        
        # 키워드 확인
        if keywords and question in keywords:
            kw_list = keywords[question]
            base_kw_count = sum(1 for kw in kw_list if kw.lower() in base_answer.lower())
            ft_kw_count = sum(1 for kw in kw_list if kw.lower() in ft_answer.lower())
            result["base_keyword_score"] = f"{base_kw_count}/{len(kw_list)}"
            result["ft_keyword_score"] = f"{ft_kw_count}/{len(kw_list)}"
        
        results.append(result)
    
    return pd.DataFrame(results)

# 함수 설명
print("compare_models 함수 사용법:")
print("=" * 60)
print("""
test_questions = [
    "EDA의 목적은 무엇인가요?",
    "과적합을 방지하는 방법은?"
]

# 키워드 정의 (선택)
keywords = {
    "EDA의 목적은 무엇인가요?": ["탐색", "패턴", "시각화"],
    "과적합을 방지하는 방법은?": ["정규화", "드롭아웃", "교차검증"]
}

result_df = compare_models(
    base_model=model,
    finetuned_model=finetuned_model,
    tokenizer=tokenizer,
    test_questions=test_questions,
    keywords=keywords
)

print(result_df)
""")

# 예시 출력 형식
print("\n예시 출력 DataFrame:")
sample_df = pd.DataFrame({
    "question": ["EDA의 목적은?", "과적합 방지법?"],
    "base_length": [85, 120],
    "ft_length": [156, 189],
    "base_keyword_score": ["1/3", "1/3"],
    "ft_keyword_score": ["3/3", "3/3"]
})
print(sample_df.to_string(index=False))

**풀이 설명**:
- 동일한 질문에 대해 파인튜닝 전후 모델의 응답을 비교합니다.
- **응답 길이**: 파인튜닝 후 더 상세한 응답을 생성하는지 확인
- **키워드 점수**: 중요 키워드가 응답에 포함되는지 확인
- DataFrame으로 결과를 정리하여 비교 분석이 용이하게 합니다.

---

## 학습 정리

### 핵심 개념 정리

| 개념 | 핵심 내용 | 핵심 공식/설정 |
|------|----------|---------------|
| PEFT | 일부 파라미터만 학습 | 전체의 0.1~1% |
| LoRA | 저차원 행렬 분해 | W' = W + BA |
| QLoRA | 4비트 양자화 + LoRA | 메모리 1/8 절약 |
| BitsAndBytes | 양자화 라이브러리 | NF4, Double Quant |
| SFTTrainer | LLM 파인튜닝 트레이너 | trl 라이브러리 |

### LoRA 파라미터 가이드

```python
LoraConfig(
    r=16,           # 8~64 (높을수록 표현력 증가)
    lora_alpha=32,  # r x 2 권장
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Attention
)
```

### 메모리 최적화 전략

```python
TrainingArguments(
    per_device_train_batch_size=1,      # 최소 배치
    gradient_accumulation_steps=8,      # 누적으로 효과적 배치 증가
    gradient_checkpointing=True,        # 활성화 재계산
    fp16=True                           # 16비트 학습
)
```

### 실무 체크리스트

- [ ] 모델 라이선스 확인 (상업적 사용 가능 여부)
- [ ] 데이터 포맷을 모델 템플릿에 맞게 변환
- [ ] GPU 메모리에 맞는 양자화/배치 설정
- [ ] 학습 데이터에 없는 질문으로 평가
- [ ] 어댑터 파일 저장 및 버전 관리